<a href="https://colab.research.google.com/github/sidhu2690/Deep-Learning/blob/main/04_MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [128]:
train_data = dsets.MNIST(root = './data',
                         transform = transforms.ToTensor(),
                         train = True,
                         download = True)

test_data = dsets.MNIST(root = './data',
                        transform = transforms.ToTensor(),
                        train = False,
                        )

In [129]:
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                               batch_size = 32,
                                               shuffle = True)

test_loader  = torch.utils.data.DataLoader(dataset = test_data,
                                           batch_size = 32,
                                           shuffle = False)

In [149]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 5, padding = 2, stride = 1)   # (1, 28, 28) ---> (16, 28, 28)  O = (W - K + 2P)/S + 1 = (28 - 5 + 4)/1 + 1 = 28
    self.relu = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size = 2)                                                         # (16, 28, 28)---> (16, 14, 14)  O = (W - K) / S + 1 = (28 - 2) / 2(default kernel = stride) + 1 = 14

    self.cnn2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, padding = 2, stride = 1)  # (16, 14, 14)---> (32, 14, 14)
    self.maxpool2= nn.MaxPool2d(kernel_size = 2)                                                            # (32, 14, 14)---> (32, 7, 7)

    self.linear = nn.Linear(32*7*7, 10)

  def forward(self, x):
    z = self.cnn1(x)
    z = self.relu(z)
    z = self.maxpool1(z)

    z = self.cnn2(z)
    z = self.relu(z)
    z = self.maxpool2(z)

    z = z.view(z.size(0), -1)
    z = self.linear(z)
    return z


In [150]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
epochs = 10

In [154]:
for epoch in range(epochs):
  model.train()
  loss_values = 0
  correct = 0

  for images, labels in train_loader:
    logits = model(images)
    loss = criterion(logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_values += loss.item()
    correct += (torch.argmax(logits, dim = 1) == labels).sum().item()

  accuracy = correct / len(train_loader.dataset) * 100

  print(f' Epoch: {epoch} | Loss: {loss_values / len(train_loader):.4f} | accuracy: {accuracy:.4f}')

 Epoch: 0 | Loss: 0.4246 | accuracy: 88.2467
 Epoch: 1 | Loss: 0.3372 | accuracy: 90.3683
 Epoch: 2 | Loss: 0.2943 | accuracy: 91.4683
 Epoch: 3 | Loss: 0.2636 | accuracy: 92.4000
 Epoch: 4 | Loss: 0.2390 | accuracy: 93.0250
 Epoch: 5 | Loss: 0.2188 | accuracy: 93.6400
 Epoch: 6 | Loss: 0.2015 | accuracy: 94.0783
 Epoch: 7 | Loss: 0.1861 | accuracy: 94.5950
 Epoch: 8 | Loss: 0.1734 | accuracy: 94.9783
 Epoch: 9 | Loss: 0.1620 | accuracy: 95.3150
